In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from keras_tuner.tuners import BayesianOptimization
import numpy as np
import tensorflow as tf #type: ignore
from tensorflow import keras #type: ignore
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, TensorBoard
import datetime

In [3]:
# Wczytanie danych z pliku CSV
data = pd.read_csv('../data.csv')

# Wybór 20 najliczniejszych gatunków piwa
top_20_styles = data['Style'].value_counts().nlargest(20).index
filtered_data = data[data['Style'].isin(top_20_styles)]

# Wybór tylko numerycznych kolumn do treningu
numeric_columns = ['Size(L)', 'OG', 'FG', 'ABV', 'IBU', 'Color', 'BoilSize', 'BoilTime', 'BoilGravity', 'Efficiency']
X = filtered_data[numeric_columns].copy()

# Uzupełnienie brakujących wartości średnią
X = X.fillna(X.mean())

# Przygotowanie etykiet (gatunków piwa)
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(filtered_data['Style'])

# Normalizacja danych
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Podział danych na zbiór treningowy i testowy
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Konwersja etykiet na format kategoryczny
num_classes = len(np.unique(y))
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

# Wyświetlenie kształtu zbiorów danych
print("Kształt danych treningowych:", X_train.shape)
print("Kształt danych testowych:", X_test.shape)
print("Kształt etykiet treningowych:", y_train.shape)
print("Kształt etykiet testowych:", y_test.shape)

Kształt danych treningowych: (34216, 10)
Kształt danych testowych: (8554, 10)
Kształt etykiet treningowych: (34216, 20)
Kształt etykiet testowych: (8554, 20)


In [4]:
# Listy możliwych wartości dla hiperparametrów
units_layer_1_list = [128, 160, 192, 224, 256, 288, 320, 352, 384, 416, 448, 480, 512]
units_layer_2_list = [64, 96, 128, 160, 192, 224, 256]
units_layer_3_list = [32, 48, 64, 80, 96, 112, 128]
units_layer_4_list = [16, 32, 48, 64]
dropout_list = [0.2, 0.3, 0.4, 0.5, 0.6, 0.7]
learning_rate_list = [1e-4, 3e-4, 1e-3, 3e-3, 1e-2]
batch_size_list = [16, 32, 48, 64, 80, 96, 112, 128]

# Funkcja do budowania modelu
def build_model(hp):
    model = Sequential()
    
    # Warstwa wejściowa
    model.add(Dense(
        units=hp.Choice('units_layer_1', values=units_layer_1_list),  # Wybór z listy
        activation='relu',
        input_shape=(X_train.shape[1],)
    ))
    model.add(BatchNormalization())
    model.add(Dropout(hp.Choice('dropout_layer_1', values=dropout_list)))  # Wybór z listy
    
    # Warstwa ukryta 1
    model.add(Dense(
        units=hp.Choice('units_layer_2', values=units_layer_2_list),  # Wybór z listy
        activation='relu'
    ))
    model.add(BatchNormalization())
    model.add(Dropout(hp.Choice('dropout_layer_2', values=dropout_list)))  # Wybór z listy
    
    # Warstwa ukryta 2
    model.add(Dense(
        units=hp.Choice('units_layer_3', values=units_layer_3_list),  # Wybór z listy
        activation='relu'
    ))
    model.add(BatchNormalization())
    model.add(Dropout(hp.Choice('dropout_layer_3', values=dropout_list)))  # Wybór z listy
    
    # Warstwa ukryta 3
    model.add(Dense(
        units=hp.Choice('units_layer_4', values=units_layer_4_list),  # Wybór z listy
        activation='relu'
    ))
    model.add(BatchNormalization())
    model.add(Dropout(hp.Choice('dropout_layer_4', values=dropout_list)))  # Wybór z listy
    
    # Warstwa wyjściowa
    model.add(Dense(20, activation='softmax'))  # 20 klas
    
    # Kompilacja modelu
    model.compile(
        optimizer=Adam(learning_rate=hp.Choice('learning_rate', values=learning_rate_list)),  # Wybór z listy
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model

In [6]:
tuner = BayesianOptimization(
    build_model,
    objective='val_accuracy',
    max_trials=20,
    directory='../models',
    project_name='beer_styles'
)

In [9]:
# Dodanie early stopping
early_stopping = EarlyStopping(
    monitor='val_loss',  # Monitorujemy stratę walidacyjną
    patience=5,          # Liczba epok bez poprawy po której trening zostanie zatrzymany
    restore_best_weights=True  # Przywrócenie najlepszych wag po zatrzymaniu
)

# jak cos tu bylo model_fit (jeszcze nie doszedlem dlaczego ale sie dowiem jak nie olsni bo sam to pisalem XD)
log_dir = "../models/beer_styles" #+ datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

# Trening modelu
tuner.search(
    X_train, y_train,
    epochs=20,  # Maksymalna liczba epok
    validation_split=0.2,
    callbacks=[early_stopping, tensorboard_callback],
    batch_size=32  # Możemy wybrać jedną z wartości z batch_size_list
)

Trial 20 Complete [00h 00m 44s]
val_accuracy: 0.522063136100769

Best val_accuracy So Far: 0.5244009494781494
Total elapsed time: 00h 13m 20s


In [10]:
# Pobranie najlepszego modelu
best_model = tuner.get_best_models(num_models=1)[0]

# Wyświetlenie podsumowania najlepszego modelu
best_model.summary()

c:\Users\Micha\anaconda3\Lib\site-packages\keras\src\saving\saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 38 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 416)            │         4,576 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 416)            │         1,664 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 416)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 256)            │       106,752 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 112)            │        28,784 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 112)            │           448 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 112)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 64)             │         7,232 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 64)             │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 20)             │         1,300 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 152,036 (593.89 KB)

 Trainable params: 150,340 (587.27 KB)

 Non-trainable params: 1,696 (6.62 KB)

In [11]:
# Trening najlepszego modelu
history = best_model.fit(
    X_train, y_train,
    epochs=20,  # Maksymalna liczba epok
    validation_split=0.2,
    callbacks=[early_stopping, tensorboard_callback],
    batch_size=32  # Możemy wybrać jedną z wartości z batch_size_list
)

# Wyświetlenie historii treningu
final_val_accuracy = history.history['val_accuracy'][-1]
print(f"Final Validation Accuracy: {final_val_accuracy:.2f}")

Epoch 1/20
856/856 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.4940 - loss: 1.5408 - val_accuracy: 0.5187 - val_loss: 1.4359
Epoch 2/20
856/856 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.4954 - loss: 1.5287 - val_accuracy: 0.5145 - val_loss: 1.4490
Epoch 3/20
856/856 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.4903 - loss: 1.5344 - val_accuracy: 0.5238 - val_loss: 1.4360
Epoch 4/20
856/856 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.4941 - loss: 1.5381 - val_accuracy: 0.5250 - val_loss: 1.4426
Epoch 5/20
856/856 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.4971 - loss: 1.5211 - val_accuracy: 0.5232 - val_loss: 1.4387
Epoch 6/20
856/856 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.4999 - loss: 1.5254 - val_accuracy: 0.5152 - val_loss: 1.4458
Final Validation Accuracy: 0.52


In [7]:
%load_ext tensorboard
%tensorboard --logdir ../models/beer_styles

Reusing TensorBoard on port 6006 (pid 13944), started 0:03:28 ago. (Use '!kill 13944' to kill it.)